In [3]:
import spacy_transformers

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data

# Note: this example requires the torchmetrics library: https://torchmetrics.readthedocs.io
import torchmetrics
from tqdm import tqdm

import torchhd
from torchhd import embeddings
from torchhd.models import Centroid
from torchhd.datasets import EMGHandGestures

2023-06-12 16:13:17.920967: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


ModuleNotFoundError: No module named 'torchmetrics'

In [ ]:
import pandas as pd

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using {} device".format(device))

DIMENSIONS = 10000  # number of hypervector dimensions
NUM_LEVELS = 21
BATCH_SIZE = 1  # for GPUs with enough memory we can process multiple images at ones
WINDOW = 256
N_GRAM_SIZE = 4
DOWNSAMPLE = 5
SUBSAMPLES = torch.arange(0, WINDOW, int(WINDOW / DOWNSAMPLE))


def transform(x):
    return x[SUBSAMPLES]


class Encoder(nn.Module):
    def __init__(self, out_features, timestamps, channels):
        super(Encoder, self).__init__()

        self.channels = embeddings.Random(channels, out_features)
        self.timestamps = embeddings.Random(timestamps, out_features)
        self.signals = embeddings.Level(NUM_LEVELS, out_features, high=20)

    def forward(self, input: torch.Tensor) -> torch.Tensor:
        signal = self.signals(input)
        samples = torchhd.bind(signal, self.channels.weight.unsqueeze(0))
        samples = torchhd.bind(samples, self.timestamps.weight.unsqueeze(1))

        samples = torchhd.multiset(samples)
        sample_hv = torchhd.ngrams(samples, n=N_GRAM_SIZE)
        return torchhd.hard_quantize(sample_hv)


def train(subjects=[0]):
    print("List of subjects " + str(subjects))
    ds = EMGHandGestures(
        "../data", download=True, subjects=subjects, transform=transform
    )

    train_size = int(len(ds) * 0.7)
    test_size = len(ds) - train_size
    train_ds, test_ds = data.random_split(ds, [train_size, test_size])

    train_ld = data.DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    test_ld = data.DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

    encode = Encoder(DIMENSIONS, ds[0][0].size(-2), ds[0][0].size(-1))
    encode = encode.to(device)

    num_classes = len(ds.classes)
    model = Centroid(DIMENSIONS, num_classes)
    model = model.to(device)

    with torch.no_grad():
        for samples, targets in tqdm(train_ld, desc="Training"):
            samples = samples.to(device)
            targets = targets.to(device)

            sample_hv = encode(samples)
            model.add(sample_hv, targets)

    accuracy = torchmetrics.Accuracy("multiclass", num_classes=num_classes)

    with torch.no_grad():
        model.normalize()

        for samples, targets in tqdm(test_ld, desc="Testing"):
            samples = samples.to(device)

            sample_hv = encode(samples)
            output = model(sample_hv, dot=True)
            accuracy.update(output.cpu(), targets)

    print(f"Testing accuracy of {(accuracy.compute().item() * 100):.3f}%")


Using cpu device


In [7]:
# Make a model for each subject
for i in range(5):
    experiment([i])

List of subjects [0]
Files already downloaded and verified


Testing: 100%|██████████| 158/158 [00:00<00:00, 460.32it/s]


Testing accuracy of 99.367%
List of subjects [1]
Files already downloaded and verified


Testing: 100%|██████████| 148/148 [00:00<00:00, 490.54it/s]


Testing accuracy of 97.297%
List of subjects [2]
Files already downloaded and verified


...

Testing accuracy of 99.310%
List of subjects [3]
Files already downloaded and verified


Testing: 100%|██████████| 143/143 [00:00<00:00, 419.47it/s]


Testing accuracy of 97.902%
List of subjects [4]
Files already downloaded and verified


Testing: 100%|██████████| 101/101 [00:00<00:00, 410.90it/s]

Testing accuracy of 98.020%


In [10]:
ds = EMGHandGestures(
        "../data", download=True, subjects=[0], transform=transform
    )

Files already downloaded and verified


In [14]:
pd.read_csv('../data/EMG_based_hand_gesture/COMPLETE_1.csv')

,0,0.1,0.2,0.3
0,0.009406,0.000963,0.000344,0.000144
1,0.019387,0.001922,0.000728,0.000299
2,0.029962,0.002871,0.001077,0.000471
3,0.041019,0.003817,0.001441,0.000610
4,0.052452,0.004767,0.001764,0.000747
...,...,...,...,...
148825,0.305463,0.038035,0.024805,0.014317
148826,0.306352,0.037998,0.024806,0.014292
148827,0.307245,0.037950,0.024790,0.014287
148828,0.308179,0.037895,0.024767,0.014313


In [18]:
pd.read_csv('../data/EMG_based_hand_gesture/LABEL_4.csv')

,1
0,1
1,1
2,1
3,1
4,1
...,...
134329,1
134330,1
134331,1
134332,1
